## Local GloVe Setup Code

In [6]:
# Place local path of GloVe here
GloVe_path = 'data\glove.6B.50d.txt'

In [7]:
with open(GloVe_path) as f:
    print(f.readline(10))

the 0.418 


## Google Drive Setup Code

In [ ]:
# Place path of GloVe here
GloVe_path = '/gdrive/My Drive/Projects/Bumblebee/using-pretrained-glove-vectors-example/glove.6B.50d.txt'

In [4]:
from google.colab import drive
drive.mount('/gdrive')

ModuleNotFoundError: No module named 'google'

In [ ]:
with open('/gdrive/My Drive/Projects/Bumblebee/using-pretrained-glove-vectors-example/glove.6B.50d.txt', 'r') as f:
  print(f.readline(10))

the 0.418 


In [ ]:
# import numpy as np
# # from numpy import arccos
# from scipy import spatial
# # import matplotlib.pyplot as plt
# # from sklearn.manifold import TSNE
# # from sklearn.metrics.pairwise import cosine_similarity
# # from collections import OrderedDict
# import re

## Package Import

In [22]:
import re
import numpy as np
from scipy import spatial

In [55]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize

In [54]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Kevin Yang\anaconda3\envs\Voryx\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Kevin Yang\anaconda3\envs\Voryx\lib\site-packages\nltk\draw\table.py", line 198, in _resize_column_motion_cb
    lb["width"] = max(3, lb["width"] + (x1 - x2) // charwidth)
  File "C:\Users\Kevin Yang\anaconda3\envs\Voryx\lib\tkinter\__init__.py", line 1657, in __setitem__
    self.configure({key: value})
  File "C:\Users\Kevin Yang\anaconda3\envs\Voryx\lib\tkinter\__init__.py", line 1646, in configure
    return self._configure('configure', cnf, kw)
  File "C:\Users\Kevin Yang\anaconda3\envs\Voryx\lib\tkinter\__init__.py", line 1636, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
_tkinter.TclError: expected integer but got "19.0"


True

Load GloVe into dictionary for embeddings

In [24]:
embeddings_dict = {}
with open(GloVe_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

In [25]:
phrases = {
          "hi there" : "video1",  
          "how you doing" : "video2",
          "who are you" : "video3",
          "i am you" : "video3",
          "my name is josh" : "video3",
          "i am kevin" : "video3",
          "i am josh" : "vid4",
          "i josh am" : "vid5",
          "who is kevin" : "joe"
}

Class Vorx

In [60]:
class Vorx:

  def __init__(self, phrase_pool, phrase, embeddings_dict, size=15):
    self.phrase_pool = phrase_pool
    self.embeddings_dict = embeddings_dict
    self.phrase = phrase
    self.size = size
    self.embedded_phrase_pool = self.vector_sort(self.embed_phrases()[0])
    self.embedded_phrase_mapping = self.embed_phrases()[1]

  def __repr__(self):
    counter = 0

    output = ""
    output += "Sorted Embedded Phrase Pool:"
    output += "\n"

    for line in self.embedded_phrase_pool:
      phrase = self.embedded_phrase_mapping[line]

      output += str(counter) + "."
      output += "\t"
      output += phrase
      output += "\t -> \t"
      output += str(line)
      output += "\n"
    
      counter += 1
    return output

  def embed_sentence(self, sentence):
    # O(n)
    # Embed the sentence by getting the embed for each word
    words = sentence.split()
    embed = []
    for word in words:
      embed.append(self.embeddings_dict[word][1])
    return embed

  def length_normalize(self, words, size=15):
    # O(n)
    # Make the length of each vector the same to size by adding zeros
    length = len(words)
    output = words.copy()

    if length < self.size:
      difference = self.size - length
      for i in range(difference):
        output.append(0)
      return tuple(output)
    elif length > self.size:
        raise Exception("Input length is greater than size. Size is: " + str(self.size))
    else:
      return tuple(output)

  def embed_phrases(self):
    # Embed the phrases in phrase pool, internal
    embedded_phrases = []                     # List storing embeds
    mapping = {}                              # Dict used to map embed to phrase

    for phrase in self.phrase_pool:
      embed = self.embed_sentence(phrase)          # Getting embeding of sentence
      embed = self.length_normalize(embed, self.size)   # Make embed vector length same
    
      embedded_phrases.append(embed)
      mapping[embed] = phrase

    return embedded_phrases, mapping
  
  def vector_sort(self, vectors):
    # Sort the vectors in order from least to greatest
    return sorted(vectors, key=lambda x: x[0], reverse=False)
  
  def cosine_similarity(self, embed1, embed2):
    # O(1)
    # Gets angle between two vectors -> represents similarity
    if len(embed1) != len(embed2):
      raise Exception("Inputs are not same length. Embed1 is " + str(len(embed1)) + " while Embed2 is " + str(len(embed2)))
    return spatial.distance.cosine(embed1, embed2)


  def find_phrase(self, phrase):
    # Finds most similar phrase from pool to the inputted phrase
    # Outputs a tuple (In pool, index)
    sentence_in_question = self.embed_sentence(phrase)
    sentence_in_question = self.length_normalize(sentence_in_question, self.size)

    low = 0
    high = len(self.embedded_phrase_pool) - 1

    first_time = True

    while low <= high:
      mid = (high + low) // 2

      mid_embedding = self.embedded_phrase_pool[mid]

      if mid_embedding == sentence_in_question:
        first_time = False
        return True, mid
      # If x is greater, ignore left half
      elif mid_embedding < sentence_in_question:
        first_time = False
        low = mid + 1

      # If x is smaller, ignore right half
      else:
        first_time = False
        high = mid - 1

    if first_time:
      return False, -1
    else:
      return False, mid

  def prep_paragraph(self, paragraph):  
    proper_nouns = ProperNounExtractor(paragraph)                     # Using NLTK's proper noun tagger
    print("nouns:", proper_nouns)

    # all_words = re.split(', |\. |; |: | ', paragraph)               # Split inputted paragraph into stand-alone words
    # capitaled_words = [x for x in all_words if x[0].isupper()]      # Find words that start with capital letters

    prepared_para = re.split(', |\. |; |: ', paragraph)             # Split up paragraph by punctuation 

    for proper_noun in proper_nouns:
      for text in prepared_para:
        if proper_noun in text:
          parts = text.split(" " + proper_noun)
          for i, t in enumerate(parts):
            if t == "":
              parts[i] = proper_noun
          print(parts)
    return prepared_para, proper_nouns


stopwords

averaged_perceptron_tagger

punktron

In [ ]:
print("hello")